In [4]:
!pip3 install matplotlib
!pip3 install pandas
!pip3 install plotly

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.10/bin/python3.10 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.10/bin/python3.10 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.10/bin/python3.10 -m pip install --upgrade pip' command.


In [5]:
import math
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import datetime
import os

In [6]:
""" Description:
tldr me making a software to model how I spend money

Future Cool Stuff:
- use amazon or honey api to get projection of cost of things instead of general estimate
- optimize verbal insights
- add more complex costs like people's birthdays
- add markers for specific time peridos like semester days (140) and yearly goals
- add prediction software for investment portfolio at average interest rate
- show price drop reasons on-hover
- add functionality to account for dates
- can rework d/w/m/y system by just using a number of days as input per cost

If I ever wanted to sell this:
- I could probably make a google form that auto-uploads a google sheets converted to csv and then autoruns this code, taking in said csv

ref: https://www.desmos.com/calculator/1qrgcqb61s
subscriptions: https://www.amazon.com/gp/subscribe-and-save/manager/viewsubscriptions?ref_=nav_AccountFlyout_sns
"""

" Description:\ntldr me making a software to model how I spend money\n\nFuture Cool Stuff:\n- use amazon or honey api to get projection of cost of things instead of general estimate\n- optimize verbal insights\n- add more complex costs like people's birthdays\n- add markers for specific time peridos like semester days (140) and yearly goals\n- add prediction software for investment portfolio at average interest rate\n- show price drop reasons on-hover\n- add functionality to account for dates\n- can rework d/w/m/y system by just using a number of days as input per cost\n\nIf I ever wanted to sell this:\n- I could probably make a google form that auto-uploads a google sheets converted to csv and then autoruns this code, taking in said csv\n\nref: https://www.desmos.com/calculator/1qrgcqb61s\nsubscriptions: https://www.amazon.com/gp/subscribe-and-save/manager/viewsubscriptions?ref_=nav_AccountFlyout_sns\n"

In [24]:
# globals
saftey_modifier = 1.15  # percent
saftey_in_account = 100  # amt usd that should always be in account

hr_work_week = 40
tax_rate = 0.7

cols = ["Money", "Time", "Investing?"]  # x, y

In [31]:
def add_plots(df, init_money, constraints, monthly_investing=100, years=1):
    """ plots cost/time """
    total_daily = 0
    total_weekly = 0
    total_monthly = 0
    total_yearly = 0
    # TODO: implement buffer
    
    for name, info in constraints.items():
        cost = info[0]
        if cost > 0: # is posative
            cost *= saftey_modifier

        if info[1] == "d":
            total_daily += cost
        elif info[1] == "w":
            total_weekly += cost
        elif info[1] == "m":
            total_monthly += cost
        elif info[1] == "y":
            total_yearly += cost
        else:
            raise Exception("bad cost-time period calculation")
            
    start_date = datetime.datetime.now()
    end_date = start_date + datetime.timedelta(days=365*years)
    delta = datetime.timedelta(days=1)
    curr_money = init_money
    curr_money_invest = init_money
    counter = 1
    
    while start_date <= end_date:
        curr_money -= total_daily
        curr_money_invest -= total_daily
        if counter%7 == 0:
            curr_money -= total_weekly
            curr_money_invest -= total_weekly
        if counter%30 == 0:
            curr_money -= total_monthly
            curr_money_invest -= total_monthly+monthly_investing
        if counter%365 == 0:
            curr_money -= total_yearly
            curr_money_invest -= total_yearly
            
        add_df = pd.DataFrame([[curr_money, start_date, False], [curr_money_invest, start_date, True]],
                              columns=cols)
        df.append(add_df, ignore_index = True)
        start_date += delta
        counter += 1
    return df
    
    
def dup_savings(df, monthly_savings=100):
    """ adds another line into data of what my budget would look like if I invested monthly_savings """
    # TODO
    

def costs_of_note(df):
    """ prints warnings based on plotted data"""
    # TODO
    

In [32]:
if __name__ == '__main__':
    print("Starting...\n")
    
    init_data = {cols[0]: [], cols[1]: []}
    df = pd.DataFrame(init_data)
    
    # init current account amount in present day
    init_money = 350
    
    # add highlighted line of account below $100 as red zone
    
    # get income
    curr_income = float(os.environ.get("CURR_INCOME"))  # should probably do this for legal reasons
    if curr_income == None:
        print("Note: could not properly find current income")
        curr_income = 30  # template amount
    
    constraints = {
        # name: [cost, daily/weekly/monthly/yearly, next_payment_due/buffer, description]
        
        # income
        "current job": [-curr_income*hr_work_week*tax_rate, "w", 0, ""],
        
        # housing  (N/A living at home)
        
        # food
        "general food": [40, "d", 0, "approx $15 lunch, $25 dinner"],
        "soylent": [34/3, "m", 0, "coffee/cheap breakfast"],
        
        # health
        "meds": [(9/4)+24, "m", 0, "vitamins + monthly meds"],
        "hair": [120/2.5, "m", 0, "nappstar"],
        "kickboxing": [(54+2)/2, "w", 0, "biweekly class + water"],
        
        # entertainment
        "anything budget": [45, "w", 0, "money to spend on anything (extra food/cloths/ubers)"],
        "nails": [42/2, "m", 0, "getting nails/eyebrows done"],
        "anime": [8, "m", 0, "crunchyroll subscription"],
        
        # cat
        "cat_food": [41/3, "m", 0, "cat food every 3 months: https://www.amazon.com/dp/B06XDP4H17/ref=sns_myd_detail_page"],
        "cat_litter": [24/3, "m", 0, "cat litter every 3 months: https://www.amazon.com/dp/B06ZZ4679J/ref=sns_myd_detail_page"]
        # cat toys filled under extra costs
    }
    
    print("Generating Dates...\n")
    df = add_plots(df, init_money, constraints)
    
    df = dup_savings(df)
    
    df = df.sort_values(by="Time")  # ensure date sorting
    
    print("Projecting...") # plot graph
    df.graph()
    fig = px.line(df, x="Time", y="Money", color='invest?')
    
    
    # show upcoming costs over a certain amount
    print("\nCost Insights...")
    costs_of_note(df)

Starting...

Generating Dates...



ValueError: 2 columns passed, passed data had 3 columns